In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from requests.exceptions import Timeout
import time

In [6]:
#url = 'https://httpbin.io/anything' 
url = 'https://arxiv.org/list/cs/recent?show=100'
apikey = '14d2f2062a095a7caa05e0142ce147f5174b0a7e' 
params = { 'url': url, 'apikey': apikey, } 
response = requests.get('https://api.zenrows.com/v1/', params=params) 

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
title_list=[]
subjects_list=[]
authors_list=[]

infos = soup.dl.find_all('dd')

for info in infos:
    title = info.find('div', {'class': 'list-title mathjax'}).text.strip().replace('Title: ','')
    subjects = info.find('div', {'class': 'list-subjects'}).text.strip().replace('Subjects: ','')
    authors = info.find('div', {'class': 'list-authors'}).text.strip().replace('\n', '').replace('Authors:','')
    title_list.append(title)
    subjects_list.append(subjects)
    authors_list.append(authors)    

In [15]:
abstracts_list=[]

papers = soup.dl.find_all('dt')
pbar = tqdm(total=len(papers))

for paper in papers:
    detail_link = paper.find('a', {'title': 'Abstract'})['href']
    detail_url = f"https://arxiv.org{detail_link}"
#     detail_response = requests.get(detail_url, headers=headers)

#     detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
#     abstract = detail_soup.find('blockquote', {'class': 'abstract mathjax'}).text.strip().replace('\n','')
#     abstracts_list.append(abstract)
#     pbar.update(1)
#     time.sleep(3)
    
    try:
        params = { 'url': detail_url, 'apikey': apikey, }
        detail_response = requests.get('https://api.zenrows.com/v1/', params=params, timeout=10)
        
#         if detail_response.status_code == 200:
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        abstract = detail_soup.find('blockquote', {'class': 'abstract mathjax'}).text.strip()
        abstracts_list.append(abstract)
        time.sleep(3)
        pbar.update(1)
    except Timeout:
        abstracts_list.append("pass")
pbar.close()

100%|██████████| 100/100 [07:59<00:00,  4.80s/it]


In [16]:
df = pd.DataFrame({'Title': title_list, 'Subjects': subjects_list, 'Author(s)': authors_list, 'Abstract': abstracts_list})
df.to_csv('output.csv', index=False)

In [17]:
len(abstracts_list)

100